In [20]:
pip install lightgbm

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.4 MB 991.0 kB/s eta 0:00:02
   ----- ---------------------------------- 0.2/1.4 MB 2.5 MB/s eta 0:00:01
   --------------- ------------------------ 0.6/1.4 MB 4.5 MB/s eta 0:00:01
   ---------------------------------- ----- 1.2/1.4 MB 7.1 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 7.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [25]:
import pandas as pd

train_df = pd.read_csv(r".\data\playground-series-s4e6\train.csv")
test_df = pd.read_csv(r".\data\playground-series-s4e6\test.csv")

train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76518 entries, 0 to 76517
Data columns (total 38 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   id                                              76518 non-null  int64  
 1   Marital status                                  76518 non-null  int64  
 2   Application mode                                76518 non-null  int64  
 3   Application order                               76518 non-null  int64  
 4   Course                                          76518 non-null  int64  
 5   Daytime/evening attendance                      76518 non-null  int64  
 6   Previous qualification                          76518 non-null  int64  
 7   Previous qualification (grade)                  76518 non-null  float64
 8   Nacionality                                     76518 non-null  int64  
 9   Mother's qualification                 

In [9]:
train_df

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0,1,1,1,9238,1,1,126.0,1,1,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,2
1,1,1,17,1,9238,1,1,125.0,1,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,0
2,2,1,17,2,9254,1,1,137.0,1,3,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,0
3,3,1,1,3,9500,1,1,131.0,1,19,...,0,8,11,7,12.820000,0,11.1,0.6,2.02,1
4,4,1,1,2,9500,1,1,132.0,1,19,...,0,7,12,6,12.933333,0,7.6,2.6,0.32,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76513,76513,1,17,1,9254,1,1,121.0,1,19,...,0,6,8,5,10.600000,0,13.9,-0.3,0.79,2
76514,76514,1,1,6,9254,1,1,125.0,1,1,...,0,6,9,6,13.875000,0,9.4,-0.8,-3.12,2
76515,76515,5,17,1,9085,1,1,138.0,1,37,...,0,5,8,5,11.400000,1,9.4,-0.8,-3.12,1
76516,76516,1,1,3,9070,1,1,136.0,1,38,...,0,6,0,0,0.000000,0,7.6,2.6,0.32,0


In [2]:
# Check the amount of occurrences of the target variable
train_df.value_counts('Target')

Target
Graduate    36282
Dropout     25296
Enrolled    14940
Name: count, dtype: int64

In [3]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
le = LabelEncoder()

# Fit and transform the 'Target' column
train_df['Target'] = le.fit_transform(train_df['Target'])

# The mapping of original categories to integer labels
target_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print("Mapping of categories to integers:", target_mapping)

Mapping of categories to integers: {'Dropout': 0, 'Enrolled': 1, 'Graduate': 2}


In [4]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

# For example, to check the correlation of 'Admission grade' with the target
model = ols('Q("Admission grade") ~ C(Target)', data=train_df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)

                 sum_sq       df           F  PR(>F)
C(Target)  3.983974e+05      2.0  1305.28424     0.0
Residual   1.167691e+07  76515.0         NaN     NaN


In [5]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
import pandas as pd

# Initialize a list to store results
results = []

# Loop over each column in the dataframe
for column in train_df.columns:
    # Exclude 'Target' and 'id' columns
    if column not in ['Target', 'id']:
        # Check if the column is numeric
        if pd.api.types.is_numeric_dtype(train_df[column]):
            # Prepare the formula string
            formula = f'Q("{column}") ~ C(Target)'
            # Fit the model
            model = ols(formula, data=train_df).fit()
            # Perform ANOVA
            anova_result = sm.stats.anova_lm(model, typ=2)
            # Append the results, including the column name and its p-value from ANOVA
            results.append((column, anova_result.loc['C(Target)', 'PR(>F)']))

# Convert list of results into a DataFrame
results_df = pd.DataFrame(results, columns=['Variable', 'P-Value'])

# Sort the results by the p-value to see which variables are most significantly associated with the target
sorted_results = results_df.sort_values(by='P-Value')

# Print sorted results
print(sorted_results)

                                          Variable        P-Value
17                                          Gender   0.000000e+00
22             Curricular units 1st sem (enrolled)   0.000000e+00
16                         Tuition fees up to date   0.000000e+00
15                                          Debtor   0.000000e+00
23          Curricular units 1st sem (evaluations)   0.000000e+00
13                                       Displaced   0.000000e+00
12                                 Admission grade   0.000000e+00
24             Curricular units 1st sem (approved)   0.000000e+00
25                Curricular units 1st sem (grade)   0.000000e+00
28             Curricular units 2nd sem (enrolled)   0.000000e+00
29          Curricular units 2nd sem (evaluations)   0.000000e+00
30             Curricular units 2nd sem (approved)   0.000000e+00
6                   Previous qualification (grade)   0.000000e+00
31                Curricular units 2nd sem (grade)   0.000000e+00
3         

In [16]:
top_10_variables = [variable for variable in sorted_results['Variable']]

In [17]:
from sklearn.model_selection import train_test_split

X = train_df[top_10_variables]
y = train_df['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize the RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
clf.fit(X_train, y_train)

# Predict the Test set results
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

Accuracy: 0.8293910088865656
              precision    recall  f1-score   support

           0       0.90      0.83      0.86      5028
           1       0.65      0.60      0.62      3017
           2       0.85      0.92      0.89      7259

    accuracy                           0.83     15304
   macro avg       0.80      0.78      0.79     15304
weighted avg       0.83      0.83      0.83     15304



In [21]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import lightgbm as lgb

# Define feature matrix X and target vector y
X = train_df.drop(columns=['Target'])
y = train_df['Target']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the LightGBM classifier
clf = lgb.LGBMClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predict the test set results
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002335 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1629
[LightGBM] [Info] Number of data points in the train set: 61214, number of used features: 37
[LightGBM] [Info] Start training from score -1.105333
[LightGBM] [Info] Start training from score -1.635907
[LightGBM] [Info] Start training from score -0.746287
Accuracy: 0.832919498170413
              precision    recall  f1-score   support

           0       0.90      0.83      0.87      5028
           1       0.65      0.62      0.64      3017
           2       0.86      0.92      0.89      7259

    accuracy                           0.83     15304
   macro avg       0.80      0.79      0.80     15304
weighted avg       0.83      0.83      0.83     153

In [22]:
test_df = test_df[top_10_variables]
test_df

,Gender,Curricular units 1st sem (enrolled),Tuition fees up to date,Debtor,Curricular units 1st sem (evaluations),Displaced,Admission grade,Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 2nd sem (enrolled),...,Unemployment rate,Mother's occupation,Curricular units 2nd sem (credited),Inflation rate,Curricular units 1st sem (credited),Father's occupation,Father's qualification,Nacionality,International,Educational special needs
0,0,7,0,0,0,0,152.1,0,0.000000,8,...,13.9,2,0,-0.3,0,3,1,1,0,0
1,0,6,1,0,7,0,116.5,6,14.857143,6,...,11.1,4,0,0.6,0,5,19,1,0,0
2,0,6,1,0,11,0,114.2,6,12.000000,6,...,15.5,4,0,2.8,0,9,19,1,0,0
3,0,6,1,0,15,0,130.0,5,11.500000,8,...,8.9,3,3,1.4,2,3,19,1,0,0
4,0,6,1,0,9,1,106.0,3,11.000000,6,...,7.6,5,0,2.6,0,5,37,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51007,0,0,1,0,0,1,124.7,0,0.000000,0,...,15.5,7,0,2.8,0,10,37,1,0,0
51008,1,5,0,1,6,0,140.0,0,0.000000,5,...,9.4,9,0,-0.8,0,9,37,1,0,0
51009,0,0,0,1,0,0,120.4,0,0.000000,0,...,15.5,4,0,2.8,0,10,1,1,0,0
51010,0,6,0,1,8,1,126.3,5,12.600000,6,...,7.6,5,0,2.6,0,5,19,1,0,0


In [26]:
# Create submission

test_predictions = clf.predict(test_df)

test_predictions = [prediction for prediction in test_predictions]


# Revert log transformation for "Rings" variable
test_df = pd.read_csv(r".\data\playground-series-s4e6\test.csv")

list_of_ids = [id for id in test_df['id']]

test_predictions

[0,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 1,
 0,
 1,
 2,
 2,
 1,
 2,
 2,
 0,
 2,
 1,
 0,
 0,
 0,
 2,
 2,
 1,
 2,
 2,
 0,
 1,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 0,
 0,
 1,
 2,
 2,
 2,
 0,
 0,
 2,
 1,
 1,
 1,
 2,
 2,
 1,
 2,
 0,
 2,
 0,
 2,
 1,
 0,
 2,
 1,
 1,
 2,
 2,
 2,
 0,
 1,
 2,
 2,
 1,
 0,
 0,
 2,
 2,
 0,
 0,
 2,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 0,
 0,
 1,
 2,
 2,
 2,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 2,
 1,
 2,
 2,
 2,
 1,
 2,
 0,
 2,
 2,
 2,
 1,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 1,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 1,
 2,
 0,
 1,
 2,
 0,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 1,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 1,
 2,
 0,
 0,
 1,
 1,
 2,
 2,
 0,
 2,
 2,
 0,
 1,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 2,
 1,
 0,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 0,
 2,
 1,
 2,
 0,
 1,
 0,
 0,
 2,
 1,
 1,
 0,
 0,
 2,
 0,
 2,
 2,
 2,


In [27]:
inverse_target_mapping = {v: k for k, v in target_mapping.items()}

mapped_predictions = [inverse_target_mapping[prediction] for prediction in test_predictions]

mapped_predictions

['Dropout',
 'Graduate',
 'Graduate',
 'Enrolled',
 'Enrolled',
 'Graduate',
 'Graduate',
 'Graduate',
 'Dropout',
 'Graduate',
 'Graduate',
 'Graduate',
 'Graduate',
 'Enrolled',
 'Dropout',
 'Enrolled',
 'Graduate',
 'Graduate',
 'Enrolled',
 'Graduate',
 'Graduate',
 'Dropout',
 'Graduate',
 'Enrolled',
 'Dropout',
 'Dropout',
 'Dropout',
 'Graduate',
 'Graduate',
 'Enrolled',
 'Graduate',
 'Graduate',
 'Dropout',
 'Enrolled',
 'Graduate',
 'Dropout',
 'Dropout',
 'Graduate',
 'Graduate',
 'Graduate',
 'Graduate',
 'Graduate',
 'Graduate',
 'Graduate',
 'Enrolled',
 'Graduate',
 'Dropout',
 'Dropout',
 'Enrolled',
 'Graduate',
 'Graduate',
 'Graduate',
 'Dropout',
 'Dropout',
 'Graduate',
 'Enrolled',
 'Enrolled',
 'Enrolled',
 'Graduate',
 'Graduate',
 'Enrolled',
 'Graduate',
 'Dropout',
 'Graduate',
 'Dropout',
 'Graduate',
 'Enrolled',
 'Dropout',
 'Graduate',
 'Enrolled',
 'Enrolled',
 'Graduate',
 'Graduate',
 'Graduate',
 'Dropout',
 'Enrolled',
 'Graduate',
 'Graduate',
 'En

In [28]:
import csv

# Writing data to CSV file
with open('output.csv', 'w', newline='') as csvfile:
    fieldnames = ['id', 'Target']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for id, rings in zip(list_of_ids, mapped_predictions):
        writer.writerow({'id': id, 'Target': rings})

print("CSV file 'output.csv' has been created successfully.")

CSV file 'output.csv' has been created successfully.
